[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/reznikmm/ada-howto/advent-2020?filepath=%2Fhome%2Fjovyan%2Fnb%2F23%2F23.ipynb)

# Day 23

We will use a lot of space in the primary stack. Let's increase default stack size with next magic by specifying a linker option:

In [1]:
%largs -Wl,--defsym=__stack_size=130000000

Now -largs options are: -Wl,--defsym=__stack_size=130000000

We need some standard packages to write result and keep cups.

In [2]:
with Ada.Text_IO;
with Ada.Containers.Doubly_Linked_Lists;

Now let's declare a type for Cup label. Also we need a list of cups to represent cup circle. An index array to speed up cup search in the circle.

In [3]:
type Cup is range 1 .. 1_000_000;

function Prev (Value : Cup) return Cup is
  (if Value = Cup'First then Cup'Last else Value - 1);

package Cup_Lists is new Ada.Containers.Doubly_Linked_Lists (Cup);

type Index_Array is array (Cup) of Cup_Lists.Cursor;

Data    : Cup_Lists.List;
Index   : Index_Array;

A subprogram to pick cup up from the list into `Value`.

In [4]:
procedure Pick
  (Cursor : in out Cup_Lists.Cursor;
   Value  : out Cup);
--  Delete a cup from Data list and move Cursor

procedure Pick
  (Cursor : in out Cup_Lists.Cursor;
   Value  : out Cup)
is
   Next : constant Cup_Lists.Cursor := Cup_Lists.Next (Cursor);
begin
   Value := Cup_Lists.Element (Cursor);
   Data.Delete (Cursor);

   if Cup_Lists.Has_Element (Next) then
      Cursor := Next;
   else
      Cursor := Data.First;
   end if;

   Index (Value) := Cup_Lists.No_Element;
end Pick;

Another subprogram to put cup back to the circle after given position

In [5]:
procedure Put
  (Value : Cup;
   After : in out Cup_Lists.Cursor);
--  Put Value back into Data list after given cursor

procedure Put
  (Value : Cup;
   After : in out Cup_Lists.Cursor)
is
   Before : Cup_Lists.Cursor := Cup_Lists.Next (After);
begin
   if Cup_Lists.Has_Element (Before) then
      Data.Insert (Before, Value, Position => After);
   else
      Data.Append (Value);
      After := Data.Last;
   end if;

   Index (Value) := After;
end Put;

Initial values and the current cup:

In [6]:
Init : constant array (Positive range <>) of Cup :=
  (4, 6, 3, 5, 2, 8, 1, 7, 9);

Current : Cup := Init (1);

Put initial cups into `Data`

In [7]:
for J of Init loop
   Data.Append (J);
   Index (J) := Data.Last;
end loop;

An put rest of cups

In [8]:
for J in Cup'(10) .. 1_000_000 loop
   Data.Append (J);
   Index (J) := Data.Last;
end loop;

We are ready to play game

In [9]:
for J in 1 .. 10_000_000 loop
   declare
      Up     : array (1 .. 3) of Cup;
      Cursor : Cup_Lists.Cursor := Cup_Lists.Next (Index (Current));
      Dest   : Cup := Prev (Current);
   begin
      if not Cup_Lists.Has_Element (Cursor) then
         Cursor := Data.First;
      end if;
         
      for Item of Up loop
         Pick (Cursor, Item);
      end loop;

      Current := Cup_Lists.Element (Cursor);

      loop
         exit when (for all Item of Up => Item /= Dest);
         Dest := Prev (Dest);
      end loop;

      Cursor := Index (Dest);

      for Item of Up loop
         Put (Item, Cursor);
      end loop;
   end;
end loop;

Now just find cups and print them

In [10]:
declare
   Cursor : Cup_Lists.Cursor := Cup_Lists.Next (Index (1));
   Cup_1  : Cup := Cup_Lists.Element (Cursor);
   Cup_2  : Cup := Cup_Lists.Element (Cup_Lists.Next (Cursor));
begin
   Ada.Text_IO.Put_Line (Cup_1'Image);
   Ada.Text_IO.Put_Line (Cup_2'Image);
end;

 516461
 16374


----
[Back to Table of Contents](https://github.com/reznikmm/ada-howto/tree/advent-2020) 